# Questions 3(b) SVM

## Basic Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt

# Installing libsvm
!pip install libsvm-official
from libsvm.svmutil import *

Mounted at /content/drive
  Preparing metadata (setup.py) ... done
  Created wheel for libsvm-official: filename=libsvm_official-3.32.0-cp310-cp310-linux_x86_64.whl size=123885 sha256=5f1b8ccda1f80b33b235ccb160a73afcdaab5679f0e0092984ff7d172829d25f
  Stored in directory: /root/.cache/pip/wheels/61/3b/1b/73bb4869517f96a26c82b47ccdb9ec48f12f4466de2371eff6
Successfully built libsvm-official


/usr/local/lib/python3.10/dist-packages/libsvm/svm.py:147: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def csr_to_problem_jit(l, x_val, x_ind, x_rowptr, prob_val, prob_ind, prob_rowptr, indx_start):


In [ ]:
!pip install qpsolvers
from qpsolvers import solve_qp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 2.7 MB/s eta 0:00:00


## 1.1 Binary Class SVM Using Optmization
--------------------------------------------------------

## **MAX_Margin_Classifier (SVM)**
* Construct the primal problem and find out largrangian $L(W,μ)$ and solve optmiztion problem and findout minimum value of **W** and lagrangion cofficient **μ** and put value of W in $L(W,μ)$ and then i will find out my Dual Function
$$$$
* $$
Dual\ Function:\ Q(μ)\ =\ \sum_{i=0}^{n} {μ_i}\ -\ 0.5*\sum_{i=0}^{n}\sum_{j=0}^{n} {μ_i}{μ_j}{y_i}{y_j}{X_i}^T{X_j}
$$
* $$
Dual\ Problem: max\ Q(μ)\ S.T.\ 0<=μ<=c\ \&\ \sum_{i=0}^{n} {μ_i}{y_i}\ =\ 0
$$
$$$$
* optmize above QP problem using SMO or QPSolver or we can also use directly LibSVM library for solve above problem


In [ ]:
# Load Data
data=np.loadtxt('/content/drive/My Drive/Prnn_data_group_1/binary_class_classification_data_group_0_train.txt',skiprows=1)
np.random.shuffle(data)
X,labels=data[:,:-1],data[:,-1]
labels = np.where(labels == 0, -1, 1)

In [ ]:
positives = np.sum(labels == 1)
negatives = np.sum(labels == -1)

if positives < 2000 or negatives < 2000:
    print("Warning: Not enough data points for balanced sampling. Using all available data.")
    sample_indices = np.random.choice(len(X), size=1000, replace=False)
else:
    positive_indices = np.random.choice(np.where(labels == 1)[0], size=2000, replace=False)
    negative_indices = np.random.choice(np.where(labels == -1)[0], size=2000, replace=False)

    sample_indices = np.concatenate((positive_indices, negative_indices))
    np.random.shuffle(sample_indices)

remaining_indices = np.setdiff1d(np.arange(len(data)), sample_indices)
np.random.shuffle(remaining_indices)
X_train= X[sample_indices]
y_train = labels[sample_indices]

X_val=X[remaining_indices[:5000]]
y_val=labels[remaining_indices[:5000]]
X_test = X[remaining_indices[5000:]]
y_test = labels[remaining_indices[5000:]]

X_train.shape,y_train.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

((4000, 10), (4000,), (5000, 10), (5000,), (5000, 10), (5000,))

In [ ]:
class BINARY_SVM():
  def __init__(self,X_train, X_test, y_train, y_test,input,C,degree=None,gamma=None):
    self.X_train=X_train
    self.y_train=y_train
    self.X_test=X_test
    self.y_test=y_test
    self.degree=degree
    self.gamma=gamma
    self.C=C
    self.input=input
    self.kernel=[self.linear_kernel, self.polynomial_kernel, self.rbf_kernel]
    return

  def linear_kernel(self,x1,x2):
    return np.dot(x1,x2.T)

  def polynomial_kernel(self,X1,X2):
    return (np.dot(X1, X2.T) + 1) ** self.degree

  def rbf_kernel(self,x1,x2):
    try:
      distance = np.linalg.norm(x1[:, None] - x2, axis=2) ** 2
    except:
      distance = np.linalg.norm(x1 - x2) ** 2
    return np.exp(-self.gamma * distance)

  def prediction(self):
    n_samples, n_features = self.X_train.shape
    K=self.kernel[self.input](self.X_train, self.X_train)
    P = np.outer(self.y_train, self.y_train) * K
    q = -np.ones(n_samples)
    G = np.vstack((-np.eye(n_samples), np.eye(n_samples)))
    h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C))
    A = self.y_train.reshape(1, -1).astype('d')
    b = np.zeros(1)
    mu = np.array(solve_qp(P, q, G, h ,A,b,solver='cvxopt')).flatten()

    support_vector_indices = np.where((mu > 1e-10) & (mu < self.C))
    X_support = self.X_train[support_vector_indices]
    y_support = self.y_train[support_vector_indices]
    mu_support = mu[support_vector_indices]

    w = np.sum(mu_support * y_support * X_support.T, axis=1)
    b = np.mean(y_support - np.dot(X_support, w))

    z = np.dot(self.X_test, w) + b
    y_pred = np.sign(z)
    accuracy = np.mean(y_pred == self.y_test)
    return accuracy * 100 , y_pred

## 1.1(a) Hard_BinaryClass_SVM_Using_Optmization
-----------------------------------------------------------

#### Linear_Kernel (Input=0)

In [ ]:
result=BINARY_SVM(X_train, X_val, y_train, y_val, input=0, C=100)
Accuracy , y_pred_Linear_HB_SVM = result.prediction()
print(f"Accuracy for Hard BinaryClass SVM  with Linear_kernel is on validation_data{Accuracy}%")
print(f"y_prediction 10 sample are {y_pred_Linear_HB_SVM[:10]}")

Accuracy for Hard BinaryClass SVM  with Linear_kernel is on validation_data69.24%
y_prediction 10 sample are [ 1. -1.  1. -1. -1. -1.  1. -1.  1.  1.]


In [ ]:
result=BINARY_SVM(X_train, X_test, y_train, y_test, input=0, C=100)
Accuracy , y_pred_Linear_HB_SVM = result.prediction()
print(f"Accuracy for Hard BinaryClass SVM  with Linear_kernel is on test_data{Accuracy}%")
print(f"y_prediction 10 sample are {y_pred_Linear_HB_SVM[:10]}")

Accuracy for Hard BinaryClass SVM  with Linear_kernel is on test_data68.38%
y_prediction 10 sample are [ 1. -1.  1.  1. -1.  1.  1.  1.  1. -1.]


#### Polynomial_kernel (Input=1)

In [ ]:
degree_values = np.linspace(1,5,2)
results_shape = (len(degree_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(degree_values):
    accuracy, y_pred_Poly_HB_SVM = BINARY_SVM(X_train, X_val, y_train, y_val, input=1, degree=param1_value, C=100).prediction()
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = degree_values[best_param1_idx]
print(f"Best parameters: Degree_of_polynomial = {best_param1}")
print(f"Accuracy for Hard BinaryClass SVM  with Polynomials _kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 10 sample are on validation data {y_pred_Poly_HB_SVM[:10]}")
print("")
print("TEST DATA")
print("")
result=BINARY_SVM(X_train, X_test, y_train, y_test, input=1, degree=best_param1, C=100)
Accuracy , y_pred_poly_HB_SVM = result.prediction()
print(f"Accuracy for Hard BinaryClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_poly_HB_SVM[:10]}")

Best parameters: Degree_of_polynomial = 1.0
Accuracy for Hard BinaryClass SVM  with Polynomials _kernel is on validation data67.92%
y_prediction 10 sample are on validation data [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

TEST DATA

Accuracy for Hard BinaryClass SVM  with Polynomial_kernel is on test_data 69.26%
y_prediction 10 sample are on test data [ 1.  1. -1.  1.  1.  1.  1.  1. -1. -1.]


#### RBF Kennel (Input=2)

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
results_shape = (len(gamma_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    accuracy, y_pred_Poly_HB_SVM = BINARY_SVM(X_train, X_val, y_train, y_val, input=2, gamma=param1_value, C=100).prediction()
    results[param1_idx] = accuracy

best_param1_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
print(f"Best parameters: gamma_value of Rbf kernel = {best_param1}")
print(f"Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 10 sample are on validation data {y_pred_Poly_HB_SVM[:10]}")
print("")
print("TEST DATA")
print("")
result=BINARY_SVM(X_train, X_test, y_train, y_test, input=2, gamma=best_param1, C=100)
Accuracy , y_pred_rbf_HB_SVM = result.prediction()
print(f"Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_rbf_HB_SVM[:10]}")

Best parameters: gamma_value of Rbf kernel = 0.001
Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on validation data 67.72%
y_prediction 10 sample are on validation data [-1. -1.  1.  1.  1.  1. -1.  1.  1. -1.]

TEST DATA

Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on test_data 68.97999999999999%
y_prediction 10 sample are on test data [ 1.  1.  1.  1.  1. -1.  1.  1. -1. -1.]


## 1.1(b) SOFT_BinaryClass_SVM_Using_Optmization
------------------------------------------------------------

#### Linear_Kennel (Input=0)

In [ ]:
C_values = np.logspace(-2, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, y_pred_Poly_HB_SVM = BINARY_SVM(X_train, X_val, y_train, y_val, input=0, C=param1_value).prediction()
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft BinaryClass SVM  with Linear_kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 10 sample are on validation data {y_pred_Poly_HB_SVM[:10]}")
print("")
print("TEST DATA")
print("")
result=BINARY_SVM(X_train, X_test, y_train, y_test, input=0, C=best_param1)
Accuracy , y_pred_Linear_SB_SVM = result.prediction()
print(f"Accuracy for Soft BinaryClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_Linear_SB_SVM[:10]}")

Best parameters: C_parameter = 0.05623413251903491
Accuracy for Soft BinaryClass SVM  with Linear_kernel is on validation data68.10000000000001%
y_prediction 10 sample are on validation data [-1. -1.  1.  1.  1.  1. -1.  1.  1.  1.]

TEST DATA

Accuracy for Soft BinaryClass SVM  with Linear_kernel is on test_data 69.14%
y_prediction 10 sample are on test data [ 1.  1. -1.  1.  1.  1.  1.  1. -1. -1.]


#### Polynomial _Kennel With GridSearch (Input=1)

In [ ]:
Degree_values = np.linspace(1, 5, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(Degree_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(Degree_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = BINARY_SVM(X_train, X_val, y_train, y_val, input=1, degree=param1_value ,C=param2_value).prediction()
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = Degree_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft BinaryClass SVM  with Ploynomail kernel with GridSearch is {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=BINARY_SVM(X_train, X_test, y_train, y_test, input=1, degree=best_param1, C=best_param2)
Accuracy , y_pred_poly_SB_SVM = result.prediction()
print(f"Accuracy for Soft BinaryClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_poly_SB_SVM[:10]}")



Validation DATA

Best parameters: Degree_of_polynomial = 1.0, C_value = 0.05623413251903491
Accuracy for Soft BinaryClass SVM  with Ploynomail kernel with GridSearch is 68.54%

TEST DATA

Accuracy for Soft BinaryClass SVM  with Polynomial_kernel is on test_data 68.84%
y_prediction 10 sample are on test data [-1. -1.  1. -1. -1.  1.  1. -1.  1.  1.]


#### RBF Kennel With GridSearch (Input=2)

In [ ]:
gamma_values = np.logspace(-4, 0, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(gamma_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = BINARY_SVM(X_train, X_val, y_train, y_val, input=2, gamma=param1_value ,C=param2_value).prediction()
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Gamma_value in Rbf Kernel = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft BinaryClass SVM  with Rbf kernel with GridSearch is {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=BINARY_SVM(X_train, X_test, y_train, y_test, input=2, gamma=best_param1, C=best_param2)
Accuracy , y_pred_rbf_SB_SVM = result.prediction()
print(f"Accuracy for Soft BinaryClass SVM  with rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_rbf_SB_SVM[:10]}")


Validation DATA

Best parameters: Gamma_value in Rbf Kernel = 0.001, C_value = 10.0
Accuracy for Soft BinaryClass SVM  with Rbf kernel with GridSearch is 68.78%

TEST DATA

Accuracy for Soft BinaryClass SVM  with rbf_kernel is on test_data 68.66%
y_prediction 10 sample are on test data [-1. -1. -1. -1.  1. -1. -1.  1.  1.  1.]


## 1.2 Multi Class SVM Using Optmization
------------------------------------------------------

In [ ]:
#Load Data
Multiclass_data=np.loadtxt('/content/drive/My Drive/Prnn_data_group_1/multi_class_classification_data_group_0_train.txt',skiprows=1)
np.random.shuffle(Multiclass_data)
X_data,y_data=Multiclass_data[:,:-1],Multiclass_data[:,-1]
X_multi_train,y_multi_train=X_data[:2000],y_data[:2000]
X_multi_val,y_multi_val=X_data[2000:8000],y_data[2000:8000]
X_multi_test,y_multi_test=X_data[8000:],y_data[8000:]


X_multi_train.shape,y_multi_train.shape,X_multi_val.shape,y_multi_val.shape,X_multi_test.shape,y_multi_test.shape

((2000, 25), (2000,), (6000, 25), (6000,), (62000, 25), (62000,))

In [ ]:
class MULTI_SVM():
  def __init__(self, X_train, X_test, y_train, y_test, input, C, degree=None, gamma=None):
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test
    self.degree = degree
    self.gamma = gamma
    self.C = C
    self.input = input
    self.kernel = [self.linear_kernel, self.polynomial_kernel, self.rbf_kernel]
    self.n_classes = 10

  def linear_kernel(self, x1, x2):
    return np.dot(x1, x2.T)

  def polynomial_kernel(self, X1, X2):
    return (np.dot(X1, X2.T) + 1) ** self.degree

  def rbf_kernel(self, x1, x2):
    try:
      distance = np.linalg.norm(x1[:, None] - x2, axis=2) ** 2
    except:
      distance = np.linalg.norm(x1 - x2) ** 2
    return np.exp(-self.gamma * distance)

  def prediction(self):
    n_samples, n_features = self.X_train.shape
    y_pred = np.zeros(len(self.y_test))
    for class_index in range(self.n_classes):
      y_train_binary = np.where(self.y_train == class_index, 1, -1)

      K = self.kernel[self.input](self.X_train, self.X_train)
      P = np.outer(y_train_binary, y_train_binary) * K
      q = -np.ones(n_samples)
      G = np.vstack((-np.eye(n_samples), np.eye(n_samples)))
      h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C))
      A = self.y_train.reshape(1, -1).astype('d')
      b = np.zeros(1)
      mu = np.array(solve_qp(P, q, G, h ,A,b,solver='cvxopt')).flatten()

      support_vector_indices = np.where((mu > 1e-10) & (mu < self.C))
      X_support = self.X_train[support_vector_indices]
      y_support = self.y_train[support_vector_indices]
      mu_support = mu[support_vector_indices]

      w = np.sum(mu_support * y_support * X_support.T, axis=1)
      b = np.mean(y_support - np.dot(X_support, w))

      z = np.dot(self.X_test, w) + b
      class_y_pred = np.sign(np.array(z))
      y_pred[class_y_pred == 1] = class_index

    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy * 100 , y_pred

## 1.2(a) Hard_MultiClass_SVM_Using_Optmization
------------------------------------------------------

#### Linear_kernel (Input=0)

In [ ]:
print("")
print("Validation Data")
print("")
model = MULTI_SVM(X_multi_train, X_multi_val, y_multi_train, y_multi_val, input=0, C=100)
Accuracy , y_pred_Linear_HM_SVM = model.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Linear_HM_SVM[:20]}")
print("")
print("Test Data")
print("")
model = MULTI_SVM(X_multi_train, X_multi_test, y_multi_train, y_multi_test, input=0, C=100)
Accuracy , y_pred_Linear_HM_SVM = model.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Linear_HM_SVM[:20]}")


Validation Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data9.857142857142858%
y_prediction 20 sample are [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Test Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data 10.043548387096775%
y_prediction 20 sample are [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### polynomial_kernel (Input=1)

In [ ]:
degree_values = np.linspace(0,5,6)
results_shape = (len(degree_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(degree_values):
    accuracy, y_pred_Poly_HM_SVM = MULTI_SVM(X_multi_train, X_multi_val, y_multi_train, y_multi_val, input=1, degree=param1_value, C=100).prediction()
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = degree_values[best_param1_idx]
print(f"Best parameters: Degree_of_polynomial = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM  with Polynomials _kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 20 sample are on validation data {y_pred_Poly_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(X_multi_train, X_multi_test, y_multi_train, y_multi_test, input=1, degree=best_param1, C=100)
Accuracy , y_pred_poly_HM_SVM = result.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_HM_SVM[:20]}")

Best parameters: Degree_of_polynomial = 0.0
Accuracy for Hard MultiClass SVM  with Polynomials _kernel is on validation data 9.8%
y_prediction 20 sample are on validation data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

TEST DATA

Accuracy for Hard MultiClass SVM  with Polynomial_kernel is on test_data 10.06774193548387%
y_prediction 20 sample are on test data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### Rbf_kernel (Input=2)

In [ ]:
gamma_values = np.logspace(-5, 0, 5)
results_shape = (len(gamma_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    accuracy, y_pred_rbf_HM_SVM = MULTI_SVM(X_multi_train, X_multi_val, y_multi_train, y_multi_val, input=2, gamma=param1_value, C=100).prediction()
    results[param1_idx] = accuracy

best_param1_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
print(f"Best parameters: gamma_value of Rbf kernel = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 20 sample are on validation data {y_pred_rbf_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(X_multi_train, X_multi_test, y_multi_train, y_multi_test, input=2, gamma=best_param1, C=100)
Accuracy , y_pred_rbf_HM_SVM = result.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_HM_SVM[:20]}")

Best parameters: gamma_value of Rbf kernel = 1e-05
Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data 9.916666666666666%
y_prediction 20 sample are on validation data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

TEST DATA

Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data 10.058064516129033%
y_prediction 20 sample are on test data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## 1.2(b) SOFT_MultiClass_SVM_Using_Optmization
-------------------------------------------------

#### Linear_kernel (Input=0)

In [ ]:
C_values = np.logspace(-3, 1, 10)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, y_pred_Linear_HM_SVM = MULTI_SVM(X_multi_train, X_multi_val, y_multi_train, y_multi_val,input=0, C=param1_value).prediction()
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {y_pred_Linear_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(X_multi_train, X_multi_test, y_multi_train, y_multi_test, input=0, C=best_param1)
Accuracy , y_pred_Linear_SM_SVM = result.prediction()
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Linear_SM_SVM[:20]}")


Validation DATA

Best parameters: C_parameter = 0.05994842503189409
Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data 10.45%
y_prediction 20 sample are on validation data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

TEST DATA

Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data 10.029032258064516%
y_prediction 20 sample are on test data [9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]


#### Polynomial_kernel with GridSearch (Input=1)

In [ ]:
Degree_values = [1,2,3]
C_values = np.logspace(-3, 1, 5)
results_shape = (len(Degree_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(Degree_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = MULTI_SVM(X_multi_train, X_multi_val, y_multi_train, y_multi_val, input=1, degree=param1_value ,C=param2_value).prediction()
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = Degree_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(X_multi_train, X_multi_test, y_multi_train, y_multi_test, input=1, degree=best_param1, C=best_param2)
Accuracy , y_pred_poly_SM_SVM = result.prediction()
print(f"Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_SM_SVM[:20]}")



Validation DATA

Best parameters: Degree_of_polynomial = 1, C_value = 0.01
Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data 10.383333333333333%

TEST DATA

Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data 10.04516129032258%
y_prediction 20 sample are on test data [9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]


#### RBF_kernel with GridSearch (Input=2)

In [ ]:
gamma_values = np.logspace(-5, 0, 5)
C_values = np.logspace(-3, 1, 5)
results_shape = (len(gamma_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = MULTI_SVM(X_multi_train, X_multi_val, y_multi_train, y_multi_val, input=2, gamma=param1_value ,C=param2_value).prediction()
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Gamma_value in Rbf Kernel = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(X_multi_train, X_multi_test, y_multi_train, y_multi_test, input=2, gamma=best_param1, C=best_param2)
Accuracy , y_pred_rbf_SM_SVM = result.prediction()
print(f"Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_SM_SVM[:20]}")


Validation DATA

Best parameters: Gamma_value in Rbf Kernel = 0.05623413251903491, C_value = 0.001
Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data 10.45%

TEST DATA

Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data 10.029032258064516%
y_prediction 20 sample are on test data [9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]


## 2.1 BinaryClass SVM Using LIBSVM
---------------------------------------

## 2.1(a) Hard BinaryClass SVM Using LIBSVM
------------------------------------

In [ ]:
class HARD_SVM():
  def __init__(self,X_train,X_test,y_train,y_test,c):
    self.X_train=X_train
    self.X_test=X_test
    self.y_train=y_train
    self.y_test=y_test
    self.c=c
    return
  def linear_kernal(self):
    train=svm_train(self.y_train,self.X_train, f'-t 0 -c {self.c}')
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy* 100, y_pred
  def poly_kernal(self,param1):
    train=svm_train(self.y_train,self.X_train, f'-t 1 -d {int(param1)} -c {self.c}')
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred
  def Rbf_kernal(self, param1):
    train=svm_train(self.y_train,self.X_train, f'-t 2 -g {param1} -c {self.c}')
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred
  def Sigmoid_kernal(self):
    train=svm_train(self.y_train,self.X_train, f'-t 3 -c {self.c}')
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred


#### Linear Kernal

In [ ]:
print("")
print("Validation Data")
print("")
hard_margin=HARD_SVM(X_train,X_val,y_train,y_val,c=100)
Accuracy , y_pred_Linear_HB_SVM=hard_margin.linear_kernal()
print(f"Accuracy for Hard BinaryClass SVM  with Linear_kernel is on val_data {Accuracy}%")
print(f"y_prediction 10 sample are {y_pred_Linear_HB_SVM[:10]}")
print("")
print("Test Data")
print("")
hard_margin=HARD_SVM(X_train,X_test,y_train,y_test,c=100)
Accuracy , y_pred_Linear_HB_SVM=hard_margin.linear_kernal()
print(f"Accuracy for Hard BinaryClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are {y_pred_Linear_HB_SVM[:10]}")


Validation Data

Accuracy = 69.12% (3456/5000) (classification)
Accuracy for Hard BinaryClass SVM  with Linear_kernel is on val_data 69.12%
y_prediction 10 sample are [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0]

Test Data

Accuracy = 68.58% (3429/5000) (classification)
Accuracy for Hard BinaryClass SVM  with Linear_kernel is on test_data 68.58%
y_prediction 10 sample are [-1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0]


#### Polynomial Kernal

In [ ]:
degree_values = np.linspace(1,5,2)
results_shape = (len(degree_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(degree_values):
    accuracy, y_pred_Poly_HB_SVM = HARD_SVM(X_train,X_val,y_train,y_val,c=100).poly_kernal(param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = degree_values[best_param1_idx]
print(f"Best parameters: Degree_of_polynomial = {best_param1}")
print(f"Accuracy for Hard BinaryClass SVM  with Polynomials _kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 10 sample are on validation data {y_pred_Poly_HB_SVM[:10]}")
print("")
print("TEST DATA")
print("")
result=HARD_SVM(X_train,X_test,y_train,y_test, c=100)
Accuracy , y_pred_poly_HB_SVM = result.poly_kernal(best_param1)
print(f"Accuracy for Hard BinaryClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_poly_HB_SVM[:10]}")

Accuracy = 69.08% (3454/5000) (classification)
Accuracy = 76.2% (3810/5000) (classification)
Best parameters: Degree_of_polynomial = 5.0
Accuracy for Hard BinaryClass SVM  with Polynomials _kernel is on validation data 76.2%
y_prediction 10 sample are on validation data [-1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0]

TEST DATA

Accuracy = 76.58% (3829/5000) (classification)
Accuracy for Hard BinaryClass SVM  with Polynomial_kernel is on test_data 76.58%
y_prediction 10 sample are on test data [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0]


#### Rbf Kernal

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
results_shape = (len(gamma_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    accuracy, y_pred_Poly_HB_SVM = HARD_SVM(X_train,X_val,y_train,y_val, c=100).Rbf_kernal(param1_value)
    results[param1_idx] = accuracy

best_param1_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
print(f"Best parameters: gamma_value of Rbf kernel = {best_param1}")
print(f"Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 10 sample are on validation data {y_pred_Poly_HB_SVM[:10]}")
print("")
print("TEST DATA")
print("")
result=HARD_SVM(X_train,X_test,y_train,y_test, c=100)
Accuracy , y_pred_rbf_HB_SVM = result.Rbf_kernal(best_param1)
print(f"Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_rbf_HB_SVM[:10]}")

Accuracy = 71.26% (3563/5000) (classification)
Accuracy = 77.32% (3866/5000) (classification)
Accuracy = 79.32% (3966/5000) (classification)
Accuracy = 77.6% (3880/5000) (classification)
Accuracy = 55.76% (2788/5000) (classification)
Best parameters: gamma_value of Rbf kernel = 0.1
Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on validation data 79.32000000000001%
y_prediction 10 sample are on validation data [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

TEST DATA

Accuracy = 79.3% (3965/5000) (classification)
Accuracy for Hard BinaryClass SVM  with Rbf_kernel is on test_data 79.3%
y_prediction 10 sample are on test data [-1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0]


#### Sigmoid Kernal

In [ ]:
print("")
print("Validation Data")
print("")
hard_margin=HARD_SVM(X_train,X_val,y_train,y_val,c=100)
Accuracy , y_pred_Linear_HB_SVM=hard_margin.Sigmoid_kernal()
print(f"Accuracy for Hard BinaryClass SVM  with Sigmoid_kernel is on val_data {Accuracy}%")
print(f"y_prediction 10 sample are {y_pred_Linear_HB_SVM[:10]}")
print("")
print("Test Data")
print("")
hard_margin=HARD_SVM(X_train,X_test,y_train,y_test,c=100)
Accuracy , y_pred_Linear_HB_SVM=hard_margin.Sigmoid_kernal()
print(f"Accuracy for Hard BinaryClass SVM  with Sigmoid_kernal is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are {y_pred_Linear_HB_SVM[:10]}")


Validation Data

Accuracy = 49.36% (2468/5000) (classification)
Accuracy for Hard BinaryClass SVM  with Sigmoid_kernel is on val_data 49.36%
y_prediction 10 sample are [1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0]

Test Data

Accuracy = 47.96% (2398/5000) (classification)
Accuracy for Hard BinaryClass SVM  with Sigmoid_kernal is on test_data 47.96%
y_prediction 10 sample are [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


## 2.1(b) Soft BinaryClass SVM Using LIBSVM
-------------------------------------------------------------

*   C parameter controls regularization
*   Adjust C as needed


In [ ]:
class SOFT_SVM():
  def __init__(self,X_train,X_test,y_train,y_test):
    self.X_train=X_train
    self.X_test=X_test
    self.y_train=y_train
    self.y_test=y_test
    return

  def linear_kernal(self,C):
    train=svm_train(self.y_train,self.X_train, f'-t 0 -c {C}')
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred

  def poly_kernal(self,degree,C):
    train=svm_train(self.y_train, self.X_train, f"-t 1 -d {int(degree)} -c {C}")
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred

  def Rbf_kernal(self,gamma,C):
    train=svm_train(self.y_train,self.X_train, f"-t 2 -g {gamma} -c {C}")
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred

  def Sigmoid_kernal(self,C):
    train=svm_train(self.y_train,self.X_train, f'-t 3 -c {C}')
    y_pred, _, _= svm_predict(self.y_test, self.X_test, train)
    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy*100, y_pred

#### Linear Kernal

In [ ]:
C_values = np.logspace(-2, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, y_pred_Poly_SB_SVM = SOFT_SVM(X_train, X_val, y_train, y_val,).linear_kernal(C=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]
print("")
print("Find Best Parameters from Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft BinaryClass SVM  with Linear_kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 20 sample are on validation data {y_pred_Poly_SB_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=SOFT_SVM(X_train, X_test, y_train, y_test)
Accuracy , y_pred_Linear_SB_SVM = result.linear_kernal(C=best_param1)
print(f"Accuracy for Soft BinaryClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Linear_SB_SVM[:20]}")

Accuracy = 69.28% (3464/5000) (classification)
Accuracy = 69.08% (3454/5000) (classification)
Accuracy = 68.96% (3448/5000) (classification)
Accuracy = 68.98% (3449/5000) (classification)
Accuracy = 69% (3450/5000) (classification)

Find Best Parameters from Validation DATA

Best parameters: C_parameter = 0.01
Accuracy for Soft BinaryClass SVM  with Linear_kernel is on validation data 69.28%
y_prediction 20 sample are on validation data [-1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0]

TEST DATA

Accuracy = 68.52% (3426/5000) (classification)
Accuracy for Soft BinaryClass SVM  with Linear_kernel is on test_data 68.52000000000001%
y_prediction 20 sample are on test data [-1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0]


#### Grid_Search Methode for finding best hyperparameters for Polynomial Kernal

In [ ]:
Degree_values = np.linspace(1, 5, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(Degree_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(Degree_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = SOFT_SVM(X_train, X_val, y_train, y_val).poly_kernal(degree=param1_value ,C=param2_value)
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = Degree_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Find Best Parameters from Validation DATA")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft BinaryClass SVM  with Ploynomail kernel with GridSearch is on Validation Data {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=SOFT_SVM(X_train, X_test, y_train, y_test)
Accuracy , y_pred_poly_SB_SVM = result.poly_kernal(degree=best_param1, C=best_param2)
print(f"Accuracy for Soft BinaryClass SVM  with Polynomial_kernel is on Test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_SB_SVM[:20]}")

Accuracy = 68.42% (3421/5000) (classification)
Accuracy = 69.1% (3455/5000) (classification)
Accuracy = 69.2% (3460/5000) (classification)
Accuracy = 69.06% (3453/5000) (classification)
Accuracy = 69.02% (3451/5000) (classification)
Accuracy = 67.66% (3383/5000) (classification)
Accuracy = 72.24% (3612/5000) (classification)
Accuracy = 74.2% (3710/5000) (classification)
Accuracy = 74.2% (3710/5000) (classification)
Accuracy = 74.4% (3720/5000) (classification)
Accuracy = 66.72% (3336/5000) (classification)
Accuracy = 71.28% (3564/5000) (classification)
Accuracy = 75.74% (3787/5000) (classification)
Accuracy = 77.64% (3882/5000) (classification)
Accuracy = 77.88% (3894/5000) (classification)
Accuracy = 66.76% (3338/5000) (classification)
Accuracy = 69.02% (3451/5000) (classification)
Accuracy = 74.44% (3722/5000) (classification)
Accuracy = 77.88% (3894/5000) (classification)
Accuracy = 77.7% (3885/5000) (classification)
Accuracy = 68.84% (3442/5000) (classification)
Accuracy = 66.92% (

#### Grid_Search Methode for finding best hyperparameters for Rbf Kernal

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(gamma_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = SOFT_SVM(X_train, X_val, y_train, y_val).Rbf_kernal(gamma=param1_value ,C=param2_value)
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Find Best Parameters from Validation DATA")
print("")
print(f"Best parameters: Gamma_value in Rbf Kernel = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft BinaryClass SVM  with Rbf kernel with GridSearch is on Validation Data {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=SOFT_SVM(X_train, X_test, y_train, y_test)
Accuracy , y_pred_rbf_SB_SVM = result.Rbf_kernal(gamma=best_param1, C=best_param2)
print(f"Accuracy for Soft BinaryClass SVM  with rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 10 sample are on test data {y_pred_rbf_SB_SVM[:10]}")

Accuracy = 66.76% (3338/5000) (classification)
Accuracy = 66.76% (3338/5000) (classification)
Accuracy = 67.74% (3387/5000) (classification)
Accuracy = 68.9% (3445/5000) (classification)
Accuracy = 69.5% (3475/5000) (classification)
Accuracy = 67.7% (3385/5000) (classification)
Accuracy = 68.88% (3444/5000) (classification)
Accuracy = 69.74% (3487/5000) (classification)
Accuracy = 72.06% (3603/5000) (classification)
Accuracy = 74.94% (3747/5000) (classification)
Accuracy = 70.42% (3521/5000) (classification)
Accuracy = 73.46% (3673/5000) (classification)
Accuracy = 77.2% (3860/5000) (classification)
Accuracy = 78.62% (3931/5000) (classification)
Accuracy = 79.88% (3994/5000) (classification)
Accuracy = 71.28% (3564/5000) (classification)
Accuracy = 77.58% (3879/5000) (classification)
Accuracy = 77.6% (3880/5000) (classification)
Accuracy = 78.02% (3901/5000) (classification)
Accuracy = 76.22% (3811/5000) (classification)
Accuracy = 49.28% (2464/5000) (classification)
Accuracy = 49.28% 

#### Sigmoid Kernal

In [ ]:
C_values = np.logspace(-2, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, y_pred_sigmoid_SB_SVM = SOFT_SVM(X_train, X_val, y_train, y_val,).Sigmoid_kernal(C=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]
print("")
print("Find Best Parameters from Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft BinaryClass SVM with Sigmoid_kernal is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {y_pred_sigmoid_SB_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=SOFT_SVM(X_train, X_test, y_train, y_test)
Accuracy , y_pred_Linear_SB_SVM = result.Sigmoid_kernal(C=best_param1)
print(f"Accuracy for Soft BinaryClass SVM with Sigmoid_kernal is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Linear_SB_SVM[:20]}")

Accuracy = 67.68% (3384/5000) (classification)
Accuracy = 65.74% (3287/5000) (classification)
Accuracy = 55.56% (2778/5000) (classification)
Accuracy = 49.46% (2473/5000) (classification)
Accuracy = 48.72% (2436/5000) (classification)

Find Best Parameters from Validation DATA

Best parameters: C_parameter = 0.01
Accuracy for Soft BinaryClass SVM  with Sigmoid_kernal is on validation data 67.67999999999999%
y_prediction 20 sample are on validation data [1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0]

TEST DATA

Accuracy = 66.36% (3318/5000) (classification)
Accuracy for Soft BinaryClass SVM  with Sigmoid_kernal is on test_data 66.36%
y_prediction 20 sample are on test data [-1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0]


## 2.2 MultiClass SVM Using LIBSVM
-------------------------------------------------------------------
#### ONE VS REST Methods
* For each class (0 to 9), create a binary classification problem
* Treat one class as the positive class and the rest as the negative class.
* Train a binary SVM model using LibSVM for that class.Repeat this process for all 10 classes.

In [ ]:
Multiclass_data=np.loadtxt('/content/drive/My Drive/Prnn_data_group_1/multi_class_classification_data_group_0_train.txt',skiprows=1)
np.random.shuffle(Multiclass_data)
X_data,y_data=Multiclass_data[:,:-1],Multiclass_data[:,-1]
X_multi_train,y_multi_train=X_data[:1000],y_data[:1000]
X_multi_val,y_multi_val=X_data[1000:8000],y_data[1000:8000]
X_multi_test,y_multi_test=X_data[8000:],y_data[8000:]


X_multi_train.shape,y_multi_train.shape,X_multi_val.shape,y_multi_val.shape,X_multi_test.shape,y_multi_test.shape

((1000, 25), (1000,), (7000, 25), (7000,), (62000, 25), (62000,))

In [ ]:
np.min(y_multi_train), np.max(y_multi_train) # that is i have 10 classes [0,1,2,3,4,5,6,7,8,9]

(0.0, 9.0)

## 2.2(a) Hard MultiClass SVM Using LIBSVM
----------------------------------------------------------

In [ ]:
class HARD_SVM_Multiclass():
  def __init__(self,X_multi_train,X_multi_test,y_multi_train,y_multi_test):
    self.X_multi_train=X_multi_train
    self.X_multi_test=X_multi_test
    self.y_multi_train=y_multi_train
    self.y_multi_test=y_multi_test
    return

  def Linear_kernal_multiClass(self):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 0 -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Poly_kernal_multiClass(self, degree):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 1 -d {int(degree)} -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Rbf_kernal_multiClass(self, gamma):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 2 -g {gamma} -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Sigmoid_kernal_multiClass(self):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 3 -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)

    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

#### Linear Kenal with MultiClass

In [ ]:
model = HARD_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val)
accuracy , Y_pred_Linear_HM_SVM = model.Linear_kernal_multiClass()

model = HARD_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_Linear_HM_SVM = model.Linear_kernal_multiClass()


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data {accuracy}%")
print(f"y_prediction 20 sample are {Y_pred_Linear_HM_SVM[:20]}")
print("")
print("Test Data")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Linear_HM_SVM[:20]}")


Validation Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data 18.057142857142857%
y_prediction 20 sample are [7 4 5 7 1 5 2 1 4 4 1 1 1 7 7 9 2 7 7 2]

Test Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data 17.429032258064517%
y_prediction 20 sample are [1 2 8 2 1 1 5 5 7 2 2 2 2 0 1 2 1 5 1 2]


#### Polynomial kernal with MultiClass

In [ ]:
degree_values = np.linspace(1,5,2)
results_shape = (len(degree_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(degree_values):
    accuracy, Y_pred_Poly_HM_SVM = HARD_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val).Poly_kernal_multiClass(degree=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = degree_values[best_param1_idx]

result=HARD_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_poly_HM_SVM = result.Poly_kernal_multiClass(degree=best_param1)


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM with Polynomials _kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_Poly_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Hard MultiClass SVM with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_HM_SVM[:20]}")


Validation Data

Best parameters: Degree_of_polynomial = 5.0
Accuracy for Hard MultiClass SVM with Polynomials _kernel is on validation data 29.714285714285715%
y_prediction 20 sample are on validation data [5 0 6 5 7 0 7 9 6 7 1 0 5 7 0 2 6 7 7 5]

TEST DATA

Accuracy for Hard MultiClass SVM with Polynomial_kernel is on test_data 29.82258064516129%
y_prediction 20 sample are on test data [9 6 2 2 7 8 5 0 7 2 7 7 5 3 0 7 5 0 6 2]


Rbf Kernal with MultiClass

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
results_shape = (len(gamma_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    accuracy, Y_pred_rbf_HM_SVM = HARD_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val).Rbf_kernal_multiClass(gamma=param1_value)
    results[param1_idx] = accuracy

best_param1_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]

result=HARD_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_rbf_HM_SVM = result.Rbf_kernal_multiClass(gamma=best_param1)


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Best parameters: gamma_value of Rbf kernel = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_rbf_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_HM_SVM[:20]}")


Validation Data

Best parameters: gamma_value of Rbf kernel = 0.1
Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data 34.1%
y_prediction 20 sample are on validation data [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]

TEST DATA

Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data 34.96612903225807%
y_prediction 20 sample are on test data [5 9 2 2 8 7 5 3 6 2 7 8 5 3 1 6 1 0 6 2]


#### Sigmoid Kernal with MultiClass

In [ ]:
model = HARD_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val)
accuracy , Y_pred_Sigmoid_HM_SVM = model.Sigmoid_kernal_multiClass()

model = HARD_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_Sigmoid_HM_SVM = model.Sigmoid_kernal_multiClass()

Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Sigmoid_kernal is on validation data {accuracy}%")
print(f"y_prediction 20 sample are {Y_pred_Sigmoid_HM_SVM[:20]}")
print("")
print("Test Data")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Sigmoid_kernal is on test data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Sigmoid_HM_SVM[:20]}")


Validation Data

Accuracy for Hard MultiClass SVM  with Sigmoid_kernal is on validation data 14.657142857142858%
y_prediction 20 sample are [4 2 2 5 1 1 2 8 4 1 8 5 2 2 4 4 2 7 7 2]

Test Data

Accuracy for Hard MultiClass SVM  with Sigmoid_kernal is on test data 14.783870967741935%
y_prediction 20 sample are [3 2 2 2 1 1 2 5 7 7 2 7 5 7 1 7 2 1 4 5]


## 2.2(b) SOFT MultiClass SVM Using LIBSVM
---------------------------------------------------

In [ ]:
class SOFT_SVM_Multiclass():
  def __init__(self,X_multi_train,X_multi_test,y_multi_train,y_multi_test):
    self.X_multi_train=X_multi_train
    self.X_multi_test=X_multi_test
    self.y_multi_train=y_multi_train
    self.y_multi_test=y_multi_test
    return

  def Linear_kernal_multiClass(self, C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C} -t 0 -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Poly_kernal_multiClass(self,degree,C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C}  -t 1 -d {int(degree)}  -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Rbf_kernal_multiClass(self,gamma,C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C}  -t 2 -g {gamma}  -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Sigmoid_kernal_multiClass(self, C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C} -t 3 -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

#### Linear Kernal with MultiClass

In [ ]:
C_values = np.logspace(-2, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, Y_pred_Linear_HM_SVM = SOFT_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val).Linear_kernal_multiClass(C=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]

result=SOFT_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_Linear_SM_SVM = result.Linear_kernal_multiClass(C=best_param1)

Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_Linear_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Linear_SM_SVM[:20]}")


Validation DATA

Best parameters: C_parameter = 0.01
Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data 22.814285714285713%
y_prediction 20 sample are on validation data [6 2 6 7 7 2 2 8 9 7 1 5 5 2 9 9 2 7 8 5]

TEST DATA

Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data 23.393548387096775%
y_prediction 20 sample are on test data [3 5 2 2 6 1 5 5 7 2 6 8 5 3 8 9 7 7 4 2]


#### Grid_Search Methode for finding best hyperparameters for Polynomial Kernal MultiClass SVM

In [ ]:
Degree_values = np.linspace(1, 5, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(Degree_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(Degree_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy, Y_pred_poly_HM_SVM = SOFT_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val).Poly_kernal_multiClass(degree=param1_value ,C=param2_value)
      results[param1_idx, param2_idx] = accuracy

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = Degree_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]

result=SOFT_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_poly_SM_SVM = result.Poly_kernal_multiClass(degree=best_param1, C=best_param2)

Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_poly_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_SM_SVM[:20]}")


Validation DATA

Best parameters: Degree_of_polynomial = 4.0, C_value = 1.7782794100389228
Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data 32.57142857142858%
y_prediction 20 sample are on validation data [9 4 5 5 2 9 2 6 1 1 2 2 3 1 7 1 9 0 1 2]

TEST DATA

Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data 33.133870967741935%
y_prediction 20 sample are on test data [8 2 7 5 6 2 4 6 6 7 0 7 9 7 0 9 6 0 8 9]


#### Grid_Search Methode for finding best hyperparameters for Rbf Kernal MultiClass SVM

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(gamma_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy, Y_pred_rbf_SM_SVM  = SOFT_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val).Rbf_kernal_multiClass(gamma=param1_value ,C=param2_value)
      results[param1_idx, param2_idx] = accuracy

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]

result=SOFT_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_rbf_SM_SVM = result.Rbf_kernal_multiClass(gamma=best_param1, C=best_param2)


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Gamma_value in Rbf Kernel = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_rbf_SM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_SM_SVM[:20]}")


Validation DATA

Best parameters: Gamma_value in Rbf Kernel = 0.1, C_value = 1.7782794100389228
Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data 35.58571428571429%
y_prediction 20 sample are on validation data [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

TEST DATA

Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data 35.94032258064516%
y_prediction 20 sample are on test data [0 2 6 5 6 2 4 6 6 7 1 7 9 1 0 9 6 1 1 9]


#### Sigmoid Kernal With MultiClass

In [ ]:
C_values = np.logspace(-2, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, Y_pred_Sigmoid_HM_SVM = SOFT_SVM_Multiclass(X_multi_train, X_multi_val, y_multi_train, y_multi_val).Sigmoid_kernal_multiClass(C=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]

result=SOFT_SVM_Multiclass(X_multi_train, X_multi_test, y_multi_train, y_multi_test)
Accuracy , y_pred_Sigmoid_SM_SVM = result.Sigmoid_kernal_multiClass(C=best_param1)

Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft MultiClass SVM  with Sigmoid_kernal is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_Sigmoid_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Sigmoid_kernal is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Sigmoid_SM_SVM[:20]}")


Validation DATA

Best parameters: C_parameter = 1.7782794100389228
Accuracy for Soft MultiClass SVM  with Sigmoid_kernal is on validation data 14.099999999999998%
y_prediction 20 sample are on validation data [1 2 1 1 1 9 1 1 1 7 9 2 2 1 5 1 5 5 6 9]

TEST DATA

Accuracy for Soft MultiClass SVM  with Sigmoid_kernal is on test_data 13.985483870967741%
y_prediction 20 sample are on test data [1 7 1 5 1 0 9 5 2 7 1 2 9 1 1 7 1 2 2 9]
